The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [10]:
%%capture

import sys
sys.path.append('..')
import mock_d2l_jax as d2l

# Custom Layers

One factor behind deep learning's success
is the availability of a wide range of layers
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.
For instance, researchers have invented layers
specifically for handling images, text,
looping over sequential data,
and
performing dynamic programming.
Sooner or later, you will encounter or invent
a layer that does not exist yet in the deep learning framework.
In these cases, you must build a custom layer.
In this section, we show you how.

## (**Layers without Parameters**)

To start, we construct a custom layer
that does not have any parameters of its own.
This should look familiar if you recall our
introduction to module in :numref:`sec_model_construction`.
The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [11]:
import jax
from jax import numpy as jnp, random, grad, vmap, jit
import flax.linen as nn

class CenteredLayer(nn.Module):
    def __call__(self, X):
        return X - X.mean()

Let's verify that our layer works as intended by feeding some data through it.


In [12]:
layer = CenteredLayer()
layer(jnp.array([1.0, 2, 3, 4, 5]))

DeviceArray([-2., -1.,  0.,  1.,  2.], dtype=float32)

We can now [**incorporate our layer as a component
in constructing more complex models.**]


In [13]:
net = nn.Sequential([nn.Dense(128), CenteredLayer()])

As an extra sanity check, we can send random data
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers,
we may still see a very small nonzero number
due to quantization.


In [14]:
Y, _ = net.init_with_output(random.PRNGKey(0), random.uniform(random.PRNGKey(1), (4, 8)))
Y.mean()

DeviceArray(9.313226e-09, dtype=float32)

## [**Layers with Parameters**]

Now that we know how to define simple layers,
let's move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let's implement our own version of the  fully connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires two input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [15]:
class MyLinear(nn.Module):
    in_units: int
    units: int

    def setup(self):
        self.weight = self.param('weight', lambda key: random.normal(key, (self.in_units, self.units)))
        self.bias = self.param('bias', nn.initializers.zeros, (self.units,))

    def __call__(self, X):
        linear = jnp.matmul(X, self.weight) + self.bias
        return nn.relu(linear)

Next, we instantiate the `MyLinear` class
and access its model parameters.


In [16]:
linear = MyLinear(5, 3)
params = linear.init(random.PRNGKey(2), jnp.empty((3, 5)))
params

FrozenDict({
    params: {
        weight: DeviceArray([[ 0.8285992 , -1.5717859 , -0.3770015 ],
                     [-2.0587626 ,  0.70451105,  2.4615402 ],
                     [-0.23974419,  0.99390113, -0.34960932],
                     [ 0.8335171 , -0.3050124 , -0.6148325 ],
                     [ 2.6772847 ,  0.6453579 ,  0.5168642 ]], dtype=float32),
        bias: DeviceArray([0., 0., 0.], dtype=float32),
    },
})

We can [**directly carry out forward propagation calculations using custom layers.**]


In [17]:
linear.apply(params, random.uniform(random.PRNGKey(3), (2, 5)))

DeviceArray([[2.0675802 , 0.6543792 , 1.042995  ],
             [0.        , 1.1468135 , 0.66203034]], dtype=float32)

We can also (**construct models using custom layers.**)
Once we have that we can use it just like the built-in fully connected layer.


In [18]:
net = nn.Sequential([MyLinear(64, 8), MyLinear(8, 1)])
output, _ = net.init_with_output(random.PRNGKey(4), random.uniform(random.PRNGKey(5), (2, 64)))
output

DeviceArray([[2.3488579],
             [5.0308733]], dtype=float32)

## Summary

* We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
* Once defined, custom layers can be invoked in arbitrary contexts and architectures.
* Layers can have local parameters, which can be created through built-in functions.


## Exercises

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.


[Discussions](https://discuss.d2l.ai/t/59)
